# Information Transfer


In [3]:
from pgljupyter import *

## Signals

Complete the following model  to simulate the propagation of a signal



In [4]:
%%lpy -w 50 -a True

Delay = 5
LDelay = 10
T = 120

Axiom: R(T) I(0) A(Delay, 0)

derivation length: 150
production:
consider: R I A B

I(x) < A(d,o) :
  # produce flower
  if x == 1 : produce W   
  # continue to growth
  elif d > 0: produce A(d-1, o) 
  else:
    # produce lateral apex
    if o == 0: nproduce [ +(60) A(0,o+1) ] 
    # or lateral leaf
    else : nproduce [ +(60) /(60) ,(2) ~l ] 
    produce I(0) /(180)  A(Delay if o == 0 else LDelay, o)

R(t) --> R(t-1)



interpretation:

W --> _(0.3) ,(3) @O
I(x) --> ,(2 if x ==0 else 4) _(0.1 if x ==0 else 0.2) F

endlsystem


LsystemWidget(animate=True, derivationLength=151, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe…

### Modelling Fluxes

 - Model correctly the flux
<center>$\frac{\delta u}{\delta t} = \alpha \Delta u + \beta - \gamma u$</center>

  - Change the source of creation of the hormone in the middle internodes

In [24]:
%%lpy -w 100 -a True

transport_speed = 0.2
decay = 0.03
max_beta = 2
min_beta = 0

nbEntity = 10

min_level = 0.01
max_level = 10

def txt(val):
  return str(round(val,2))[:5]

Axiom: 
  nproduce _(2) 
  for i in range(nbEntity-1):
    nproduce I(min_level, min_beta)
  nproduce I(min_level, max_beta)

def du(u, deltau, beta):
    return u + deltau*transport_speed + beta - decay*u

derivation length: 400
production:

I(a, ba) < I(p, bp) > I(b, bb) --> I(du(p, a+b-2*p, bp), bp)
I(p, bp) > I(a, ba) --> I(du(p,a-p,bp), bp)
I(a, ba) < I(p, bp) --> I(du(p,a-p,bp), bp)

interpretation:

I(a, c) :
    alpha =  (a-min_level)/(max_level-min_level)
    a += 0.1
    produce InterpolateColors(4,3, alpha)F(3)[-(90)f(5)_(1)F(a)]F(3)

endlsystem


LsystemWidget(animate=True, derivationLength=401, is_magic=True, scene={'data': b'x\xda\x8d\x96\xcdn\xd3@\x10\…

## Subdivision curves

Example of simple interpolating scheme

In [29]:
%%lpy -w 3
from openalea.plantgl.all import *

Axiom: P(Vector2(0,0)) E P(Vector2(0,1)) E P(Vector2(1,1)) E P(Vector2(1,0)) E P(Vector2(2,0)) E P(Vector2(1,-1)) E P(Vector2(0,0))

derivation length: 3
production:

P(vl) < E > P(vr) --> E P(vl/2 + vr/2) E
P(v) --> P(v) 



interpretation:

P(vl) < E > P(vr) --> ,(5) _(0.02) OLineTo(0,vr[0],vr[1])
P(v) --> ,(3)@O(0.04)

endlsystem


LsystemWidget(derivationLength=4, is_magic=True, scene={'data': b'x\xda\x8d\x93\xcdJ\xc3@\x10\xc7\xa7\xdf1\x9a…

Implement the following scheme

 - Chaikin : $\frac{1}{4} [1 ~ 3 ~ 3  ~ 1]$ 
 - Dyn, Levin and Micchelli : $\frac{1}{8} [1 ~ 4 ~ 6 ~ 4 ~ 1]$
 - 4 points scheme : $\frac{1}{16} [-1 ~ 0 ~ 9 ~ 16 ~ 9 ~ 0 ~ -1]$